# Desafio Módulo 3 - Desaﬁo dos Filtros

## Objetivo

Criar um modelo de Convolutional Neural Network (CNN) capaz deidentiﬁcar faces em imagens.

Aqui estão os passos:

- Conjunto de Dados:
  - Utilize um conjunto de dados robusto, contendo imagens variadas de
faces. Divida-o em conjuntos de treinamento e teste.
- Pré-processamento:
  - Realize pré-processamento nas imagens, incluindo redimensionamento, normalização e eventual aumento de dados para evitar overﬁtting.
- Arquitetura da CNN:
  - Projete uma arquitetura de CNN adequada para o problema.
  - Considere camadas convolucionais, de pooling e totalmente conectadas.
  - Experimente arquiteturas conhecidas, como VGG16, ResNet, ou crie a sua.

<hr>
<section>
<a>https://www.kaggle.com/code/sifboudjellal/face-recognition-using-cnn</a>
<br>
<a>https://www.codemag.com/Article/2205081/Implementing-Face-Recognition-Using-Deep-Learning-and-Support-Vector-Machines</a>
<br>
<a>https://www.youtube.com/watch?v=uqomO_BZ44g</a>
</section>
<hr>

## Importando bibliotecas e pacotes

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

import os
import cv2

## Criando os objetos de modelo

In [ ]:
dados_treinamento = ImageDataGenerator(rescale = (1 / 255))
dados_validacao = ImageDataGenerator(rescale = (1 / 255))

## Carregando as imagens

### Caminho

In [ ]:
path_images_train = "datasets/images/train"
path_images_validation = "datasets/images/validation"

### Valor de re-escala das imagens

Devemos re-escalar, pois não temos garantia que:
* todas terão o mesmo tamanho,
* ou que estaram no mesmo formato (jpeg, png, tiff),
* ou que estarão no mesmo canal (rgb, cmyk, P&B, rgba);

In [ ]:
nova_escala = (150, 150)

### Tamanho do conjunto de treinamento

In [ ]:
tamanho_balote = 12

### Carregando as imagens e salvando em um objeto

In [ ]:
dataset_treinamento = dados_treinamento.flow_from_directory(
    path_images_train,
    target_size = nova_escala,
    batch_size = tamanho_balote,
    class_mode = "binary"
)

dataset_validacao = dados_treinamento.flow_from_directory(
    path_images_validation,
    target_size = nova_escala,
    batch_size = tamanho_balote,
    class_mode = "binary"
)

## Criando rede neural

In [ ]:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        64,
        (3, 3),
        activation = "relu",
        input_shape = (150, 150, 3)
    ),
    tf.keras.layers.MaxPool2D(2, 2),
    ####
    tf.keras.layers.Conv2D(
        128,
        (3, 3),
        activation = "relu",
    ),
    tf.keras.layers.MaxPool2D(2, 2),
    ####
    tf.keras.layers.Conv2D(
        256,
        (3, 3),
        activation = "relu",
    ),
    tf.keras.layers.MaxPool2D(2, 2),
    ####
    tf.keras.layers.Flatten(),
    ####
    tf.keras.layers.Dense(
        512,
        activation = "relu"
    ),
    ####
    tf.keras.layers.Dense(
        1,
        activation = "sigmoid"
    ),
])

## Compilando o modelo

In [ ]:
modelo.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
    metrics = [tf.keras.metrics.BinaryAccuracy()]
)

## Dados para treinamento do modelo

### Maximo de iterações e máximo de passos

In [ ]:
max_iter = 100
max_step = 10

### Verificando tipo do dataset

In [ ]:
type(dataset_treinamento)

### As classes do dataset

In [ ]:
dataset_validacao.classes

## Treinando o modelo

In [ ]:
modelo_fit = modelo.fit(
    dataset_treinamento,
    steps_per_epoch = max_step,
    epochs = max_iter,
    batch_size = tamanho_balote,
    validation_data = dados_validacao
)

## Atestando a qualidade do modelo

### Caminho para o diretório

In [ ]:
path_images_test = "datasets/images/test/"

### Verificação das classes

In [ ]:
dataset_validacao.class_indices

### Verificando a qualidade do modelo

In [ ]:
for new_image in os.listdir(path_images_test):
    curr_img = image.load_img(
        path_images_test + new_image,
        target_size = nova_escala
    )
    X = np.expand_dims(
        image.img_to_array(curr_img),
        axis = 0
    )
    images = np.vstack([X])
    
    modelo_predicao = modelo.predict(images)
    
    if modelo_predicao == 1:
        print("IT IS NOT A FACE")
    else:
        print("IT IS A FACE")